<a href="https://colab.research.google.com/github/bayileyegnabate/16-BigData/blob/main/cloud_etl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.co

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-07-10 19:36:47--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.02MB/s    in 0.2s    

2022-07-10 19:36:48 (5.02 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
# read in user_data from S3 Bucket
from pyspark import SparkFiles
url = "https://bc-dataviz-bucket11.s3.us-east-2.amazonaws.com/user_data.csv"

spark.sparkContext.addFile(url)

user_data_df = spark.read.csv(SparkFiles.get("user_data.csv"), sep=",", header=True, inferSchema=True)

In [5]:
# show dataframe
user_data_df.show()

+---+----------+---------+-----------+--------------------+--------------------+------------+
| id|first_name|last_name|active_user|      street_address|               state|    username|
+---+----------+---------+-----------+--------------------+--------------------+------------+
|  1|    Cletus|  Lithcow|      false| 78309 Riverside Way|            Virginia|   ibearham0|
|  2|       Caz|   Felgat|      false| 83 Hazelcrest Place|             Alabama|    wwaller1|
|  3|     Kerri|  Crowson|      false|      112 Eliot Pass|      North Carolina|   ichesnut2|
|  4|   Freddie|    Caghy|      false|     15 Merchant Way|            New York|     tsnarr3|
|  5|   Sadella|    Deuss|      false|    079 Acker Avenue|           Tennessee|   fwherrit4|
|  6|    Fraser|  Korneev|       true|  76084 Novick Court|           Minnesota|  fstappard5|
|  7|    Demott|   Rapson|       true|    86320 Dahle Park|District of Columbia|  lhambling6|
|  8|    Robert|    Poile|      false|1540 Manitowish Hill| 

In [6]:
# read in user_payment from S3 Bucket
from pyspark import SparkFiles
url = "https://bc-dataviz-bucket11.s3.us-east-2.amazonaws.com/user_payment.csv"

spark.sparkContext.addFile(url)

user_payment_df = spark.read.csv(SparkFiles.get("user_payment.csv"), sep=",", header=True, inferSchema=True)

## TRANSFORM

In [7]:
# read in user_payment from S3 Bucket
from pyspark import SparkFiles
url = "https://bc-dataviz-bucket11.s3.us-east-2.amazonaws.com/user_payment.csv"

spark.sparkContext.addFile(url)

user_payment_df = spark.read.csv(SparkFiles.get("user_payment.csv"), sep=",", header=True, inferSchema=True)

In [9]:
# join the two dataframes
joined_df = user_data_df.join(user_payment_df, on='username', how='inner')
joined_df.show()

+------------+---+----------+---------+-----------+--------------------+--------------------+----------+--------------------+
|    username| id|first_name|last_name|active_user|      street_address|               state|billing_id|        cc_encrypted|
+------------+---+----------+---------+-----------+--------------------+--------------------+----------+--------------------+
|   ibearham0|  1|    Cletus|  Lithcow|      false| 78309 Riverside Way|            Virginia|         1|a799fcafe47d7fb19...|
|    wwaller1|  2|       Caz|   Felgat|      false| 83 Hazelcrest Place|             Alabama|         2|a799fcafe47d7fb19...|
|   ichesnut2|  3|     Kerri|  Crowson|      false|      112 Eliot Pass|      North Carolina|         3|a799fcafe47d7fb19...|
|     tsnarr3|  4|   Freddie|    Caghy|      false|     15 Merchant Way|            New York|         4|a799fcafe47d7fb19...|
|   fwherrit4|  5|   Sadella|    Deuss|      false|    079 Acker Avenue|           Tennessee|         5|a799fcafe47d7f

In [10]:
# drop any rows with null values
dropna_df = joined_df.dropDuplicates()
dropna_df.show()

+-------------+---+----------+------------+-----------+--------------------+-------------+----------+--------------------+
|     username| id|first_name|   last_name|active_user|      street_address|        state|billing_id|        cc_encrypted|
+-------------+---+----------+------------+-----------+--------------------+-------------+----------+--------------------+
|    sdobing1y| 71|   Richart|     Doodney|       true|  074 Cordelia Plaza|      Indiana|        71|a799fcafe47d7fb19...|
|       cord20| 73|  Kathleen|       Kohen|       true|98944 Golf View P...|    Minnesota|        73|a799fcafe47d7fb19...|
|     ituvey3i|127|       Lem|    Eastwood|       true|        4 5th Avenue|        Texas|       127|a799fcafe47d7fb19...|
|     ksimane6|511|   Lurleen|  Beardsdale|       true|     5 Mosinee Point|       Kansas|       511|a799fcafe47d7fb19...|
| fgerritzenew|537|     Arlen|       Blose|       true|736 Oak Valley Place|     Michigan|       537|a799fcafe47d7fb19...|
|  gmccahilljq|7

In [11]:
# load in a sql function to use columns
from pyspark.sql.functions import col
#filter for only columns with active users
cleaned_df = dropna_df.filter(col("active_user") == True)
cleaned_df.show()

+-------------+---+----------+------------+-----------+--------------------+-------------+----------+--------------------+
|     username| id|first_name|   last_name|active_user|      street_address|        state|billing_id|        cc_encrypted|
+-------------+---+----------+------------+-----------+--------------------+-------------+----------+--------------------+
|    sdobing1y| 71|   Richart|     Doodney|       true|  074 Cordelia Plaza|      Indiana|        71|a799fcafe47d7fb19...|
|       cord20| 73|  Kathleen|       Kohen|       true|98944 Golf View P...|    Minnesota|        73|a799fcafe47d7fb19...|
|     ituvey3i|127|       Lem|    Eastwood|       true|        4 5th Avenue|        Texas|       127|a799fcafe47d7fb19...|
|     ksimane6|511|   Lurleen|  Beardsdale|       true|     5 Mosinee Point|       Kansas|       511|a799fcafe47d7fb19...|
| fgerritzenew|537|     Arlen|       Blose|       true|736 Oak Valley Place|     Michigan|       537|a799fcafe47d7fb19...|
|    ddraudeor|8

In [12]:
# create user dataframe to match active_user table
clean_user_df = cleaned_df.select(['id', 'first_name', 'last_name', 'username'])
clean_user_df.show()

+---+----------+------------+-------------+
| id|first_name|   last_name|     username|
+---+----------+------------+-------------+
| 71|   Richart|     Doodney|    sdobing1y|
| 73|  Kathleen|       Kohen|       cord20|
|127|       Lem|    Eastwood|     ituvey3i|
|511|   Lurleen|  Beardsdale|     ksimane6|
|537|     Arlen|       Blose| fgerritzenew|
|892|     Kathy|        null|    ddraudeor|
| 76|      Roby|      Pichan|     ghearl23|
|366|      Cort|        null|  amillwalla5|
|433|    Zollie| Sleightholm|  pborleacec0|
|284|      Milt|   Giacomini|     avobes7v|
|648|      Zeke|      Capron|     gatyeohz|
|680|     Peder|Thornthwaite| hwickersoniv|
|681|    Elisha|     Lanston|dpierrepontiw|
|238|  Consalve|      Givens|   bmirando6l|
|200|   Yanaton|        null|  lcafferty5j|
|351|     Rorie|        null|    hsuttle9q|
|387|   Kandace|        null|  avernallsaq|
|453|    Hailee|     Shackle|  ibrunstanck|
|702|     Brana|Degli Antoni|     wfloydjh|
|723|     Adele|        Clem|   

In [13]:
# create user dataframe to match billing_info
clean_billing_df = cleaned_df.select(['billing_id', 'street_address', 'state', 'username'])
clean_billing_df.show()

+----------+--------------------+-------------+-------------+
|billing_id|      street_address|        state|     username|
+----------+--------------------+-------------+-------------+
|        71|  074 Cordelia Plaza|      Indiana|    sdobing1y|
|        73|98944 Golf View P...|    Minnesota|       cord20|
|       127|        4 5th Avenue|        Texas|     ituvey3i|
|       511|     5 Mosinee Point|       Kansas|     ksimane6|
|       537|736 Oak Valley Place|     Michigan| fgerritzenew|
|       892| 9 Summerview Center|    Wisconsin|    ddraudeor|
|        76|162 Schmedeman Trail|      Florida|     ghearl23|
|       366|     3 Merrick Court|Massachusetts|  amillwalla5|
|       433|    21 Karstens Lane| Rhode Island|  pborleacec0|
|       284|       262 Sage Hill| South Dakota|     avobes7v|
|       648| 30287 Warbler Drive|     Missouri|     gatyeohz|
|       680|     9115 Susan Park|  Connecticut| hwickersoniv|
|       681| 7 Victoria Crossing|   California|dpierrepontiw|
|       

In [14]:
# create user dataframe to match payment_info table
clean_payment_df = cleaned_df.select(['billing_id', 'cc_encrypted'])
clean_payment_df.show()

+----------+--------------------+
|billing_id|        cc_encrypted|
+----------+--------------------+
|        71|a799fcafe47d7fb19...|
|        73|a799fcafe47d7fb19...|
|       127|a799fcafe47d7fb19...|
|       511|a799fcafe47d7fb19...|
|       537|a799fcafe47d7fb19...|
|       892|a799fcafe47d7fb19...|
|        76|a799fcafe47d7fb19...|
|       366|a799fcafe47d7fb19...|
|       433|a799fcafe47d7fb19...|
|       284|a799fcafe47d7fb19...|
|       648|a799fcafe47d7fb19...|
|       680|a799fcafe47d7fb19...|
|       681|a799fcafe47d7fb19...|
|       238|a799fcafe47d7fb19...|
|       200|a799fcafe47d7fb19...|
|       351|a799fcafe47d7fb19...|
|       387|a799fcafe47d7fb19...|
|       453|a799fcafe47d7fb19...|
|       702|a799fcafe47d7fb19...|
|       723|a799fcafe47d7fb19...|
+----------+--------------------+
only showing top 20 rows



## LOAD

In [21]:
# store environment variables
from getpass import getpass
password = getpass("Enter Password: ")

Enter Password: ··········


In [22]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://dataviz11.chlimk0s0kqy.us-east-2.rds.amazonaws.com:5432/data_class_db"
config = {"user":"postgres", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

In [23]:
# Write DataFrame to active_user table in RDS
clean_user_df.write.jdbc(url=jdbc_url, table='active_user', mode=mode, properties=config)

In [24]:
# Write dataframe to billing_info table in RDS
clean_billing_df.write.jdbc(url=jdbc_url, table='billing_info', mode=mode, properties=config)

In [25]:
# Write dataframe to payment_info table in RDS
clean_payment_df.write.jdbc(url=jdbc_url, table='payment_info', mode=mode, properties=config)